In [1]:
import os

os.chdir("..")

In [2]:
import polars as pl

from src.constants import TRAIN_SAMPLE_SIZE
from src.datatypes import BaseSchema, TrainSchema, filepaths, ExtendedSchema

In [3]:
# Aliases
S = TrainSchema
E = ExtendedSchema

train_schema = BaseSchema.__annotations__ | TrainSchema.__annotations__

In [4]:
data = pl.scan_parquet(filepaths.train_unique, schema=train_schema, cast_options=pl.ScanCastOptions(integer_cast='upcast')).head(TRAIN_SAMPLE_SIZE).collect()
data_lf = data.lazy()
data_lf = data_lf.cast({S.click_time: pl.Datetime('ms')})
data_lf = data_lf.sort(S.click_time, maintain_order=True)

In [5]:
from sklearn.model_selection import train_test_split


TEST_SIZE = 0.10

X = data_lf.drop(S.label(), S.attributed_time)
y = data_lf.select(S.label())

train, eval = train_test_split(
    data_lf.collect(),
    test_size=TEST_SIZE,
    shuffle=False,
)

train: pl.DataFrame
eval: pl.DataFrame

In [6]:
(train
 .group_by(S.ip, maintain_order=True)
 .agg(pl.col(S.label()).value_counts())
 .explode(S.label())
 .unnest(S.label())
 .pivot(index=S.ip, values='count', on=S.label())
 .select(S.ip, 'true', 'false') # always make `true`` column first
 .fill_null(0)
 .sort(pl.col('true').add(pl.col('false')), descending=True, maintain_order=True)
 .with_columns(total=pl.col('true').add(pl.col('false')))
 .with_columns(
     true_rel=pl.col('true').truediv('total').mul(100)
 )
 .filter(pl.col('total').gt(5000))
 .with_columns(
     pl.col(S.ip).cast(pl.String).add(pl.lit(' ')).add(pl.col('total').cast(pl.String)).cast(pl.Categorical)
 )
 .plot.bar(x=S.ip, y='true_rel').properties(title='Count by label')
)

alt.Chart(...)

# Session-based columns

Since rows of the same `ip` are dependant, we could derive how much sessions user previously had. One approach would be decide an average session lasts for example 10 minutes, but for different apps and users session times vary. Another approach will be applied: if time between two consecutive `click_time` is more than 15 minutes, than this marks end of session 1 and the beginning of session 2.

## Create `click_timestamp` column

In [7]:
first_train_datetime = train.select(pl.col(S.click_time).min())
display(first_train_datetime)

click_time
datetime[ms]
2017-11-06 16:00:01


In [8]:
click_timestamp = train.select(
    (pl.col(S.click_time)
     .dt.timestamp('ms')
     .sub(first_train_datetime.select(pl.col(S.click_time).dt.timestamp('ms')).row(0)[0])
    .floordiv(1000) # from ms to s
    )
    .cast(pl.UInt32)
    .alias(E.click_timestamp)
)
display(click_timestamp)
print()

click_timestamp.describe()
print()

display(click_timestamp.select(pl.col(E.click_timestamp).value_counts()))

click_timestamp
u32
0
0
0
0
0
…
239026
239026
239026


click_timestamp
struct[2]
"{203287,57}"
"{88143,50}"
"{96481,13}"
"{132251,56}"
"{9642,15}"
…
"{121747,44}"
"{113053,42}"
"{22077,18}"


In [9]:
train_extended = (train
 .hstack(click_timestamp)
 .select(
     pl.all()
     .exclude(S.attributed_time, S.label()), 
     S.attributed_time, 
     S.label()
 )
)

## Create `previous_sessions` column

Time between two sessions will be 15 minutes.

In [10]:
from typing import Annotated


duration_between_sessions: Annotated[int, 's'] = 15 * 60

In [11]:
previous_sessions = (train_extended
.sort([S.ip, S.click_time], maintain_order=True)
.select(
    S.ip,
    S.click_time,
    pl.col(E.click_timestamp)
    .diff()
    .fill_null(0) # because of the 1st row (does not have previous row)
    .ge(duration_between_sessions)
    .cum_sum()
    .over(S.ip)
    .alias(E.previous_sessions)
)
)
print("Previous sessions")
display(previous_sessions)
print()

Previous sessions


ip,click_time,previous_sessions
u32,datetime[ms],u32
1,2017-11-08 22:20:54,0
5,2017-11-07 05:06:16,0
6,2017-11-07 17:43:04,0
6,2017-11-07 23:11:47,1
6,2017-11-07 23:17:21,1
…,…,…
364773,2017-11-09 01:08:05,0
364773,2017-11-09 09:21:58,1
364775,2017-11-09 06:56:03,0


In [12]:
train_extended = (train_extended
 .sort(S.ip, S.click_time, maintain_order=True)
 .hstack(previous_sessions.drop(S.ip, S.click_time))
 .select(
     pl.all()
     .exclude(S.attributed_time, S.label()), 
     S.attributed_time, 
     S.label()
 )
)

## Create `total_sessions` column

In [13]:
print("Total sessions")
total_sessions = previous_sessions.select(S.ip, pl.max(E.previous_sessions).over(S.ip).alias('total_sessions')).with_columns(pl.col('total_sessions').add(1))
display(total_sessions)
print()

total_sessions_grouped = total_sessions.unique(maintain_order=True)
print("Total grouped sessions descriptives")
display(total_sessions_grouped.drop(S.ip).describe())

Total sessions


ip,total_sessions
u32,u32
1,1
5,1
6,34
6,34
6,34
…,…
364773,2
364773,2
364775,1



Total grouped sessions descriptives


statistic,total_sessions
str,f64
"""count""",143232.0
"""null_count""",0.0
"""mean""",16.344965
"""std""",22.239907
"""min""",1.0
"""25%""",1.0
"""50%""",4.0
"""75%""",23.0
"""max""",97.0


<b>VALUABLE INSIGHT</b><br>Only 25% of IPs had one session.

In [14]:
max_total_sessions: int = (
  total_sessions
  .drop(S.ip)
  .describe()
  .filter(pl.col("statistic").eq("max"))[E.total_sessions]
  .cast(int)[0]
)
print(f"{max_total_sessions = }")

ip_with_max_total_sessions: int = total_sessions.filter(pl.col(E.total_sessions).eq(max_total_sessions))[S.ip][0]
print(f"{ip_with_max_total_sessions = }")

(train
 .filter(pl.col(S.ip).eq(ip_with_max_total_sessions))
 .select(
     pl.col(S.click_time).min().alias("min_date"), 
     pl.col(S.click_time).max().alias("max_date"), 
  )
)

max_total_sessions = 97
ip_with_max_total_sessions = 173152


min_date,max_date
datetime[ms],datetime[ms]
2017-11-06 16:10:57,2017-11-09 10:17:20


In [15]:
high_session_threshold = 50

high_session_ips_per_label = (
 total_sessions
 .hstack(train.select(S.label()))
 .filter(pl.col(E.total_sessions).ge(high_session_threshold))
 .group_by(S.ip, maintain_order=True)
 .agg(pl.col(S.label()).value_counts())
 .explode(S.label())
 .unnest(S.label())
 .pivot(index=S.ip, values='count', on=S.label())
 .select(S.ip, 'true', 'false') # always make `true`` column first
 .fill_null(0)
 )
print(f"Label for IPs with {high_session_threshold} or more sessions")
display(high_session_ips_per_label)
print()

print(f"Label descriptives for IPs with {high_session_threshold} or more sessions")
display(high_session_ips_per_label.describe())

Label for IPs with 50 or more sessions


ip,true,false
u32,u32,u32
9,1,203
20,1,251
27,0,258
36,1,316
45,1,214
…,…,…
280280,2,172
280605,1,175
280918,2,299



Label descriptives for IPs with 50 or more sessions


statistic,ip,true,false
str,f64,f64,f64
"""count""",17549.0,17549.0,17549.0
"""null_count""",0.0,0.0,0.0
"""mean""",82148.901077,0.646305,258.570688
"""std""",54040.203138,0.919163,168.442117
"""min""",9.0,0.0,69.0
"""25%""",37828.0,0.0,147.0
"""50%""",76791.0,0.0,197.0
"""75%""",114632.0,1.0,299.0
"""max""",283787.0,7.0,1159.0


<b>VALUABLE INSIGHT</b><br>IPs with 50 or more total sessions usually do not download according to the median. This analysis is partly complete because it only looks at (`ip`, `total_session`) pairs and does not include `app`. Thus, IPs with none downloads and high session could be bots or users that do not care about advertisments products*. It could be some users stubbornly avoid downloading a particular particular app, but they are willing to download some other application.
4th quarter IPs downloaded between 2 and 7 times.

\* When gaming, watching an add will give you in-game benefits, so this could explain why someone has a lot of sessions in this short time period, given the fact you could gain some perks every 15 minutes in a particular game.


In [16]:
size_high_session_ips = len(high_session_ips_per_label)
print(f"{size_high_session_ips = }")

size_high_session_ips_rel = round(size_high_session_ips / len(train.select(S.ip).unique()), 4)
print(f"{size_high_session_ips_rel = }")
print()

size_high_session_downloads = len(total_sessions.filter(pl.col(E.total_sessions).ge(high_session_threshold)))
print(f"{size_high_session_downloads = }")

size_high_session_downloads_rel = round(size_high_session_downloads / len(total_sessions), 4)
print(f"{size_high_session_downloads_rel = }")

size_high_session_ips = 17549
size_high_session_ips_rel = 0.1225

size_high_session_downloads = 4548999
size_high_session_downloads_rel = 0.5054


<b>VALUABLE INSIGHT</b><br>There are <b>12.25%</b> IPs with total sessions of 50 or more, however such IPs contribute to <b>50.54%</b> of the total application downloads. 

In [17]:
train_extended = (train_extended
 .hstack(total_sessions.drop(S.ip))
 .select(
     pl.all()
     .exclude(S.attributed_time, S.label()), 
     S.attributed_time, 
     S.label()
 )
)